In [ ]:
import pysindy as ps

# the whole tutorial is under pysindy/examples/15_pysindy_lectures.ipynb

# Part 1

x_train = 0   # to be defined

feature_names = ['x', 'y', 'z']
opt = ps.STLSQ(threshold=1.0)
model = ps.SINDy(feature_names=feature_names, optimizer=opt)
model.fit(x_train, t=dt)
model.print()


In [ ]:
# Part 2: loop over different lambda and get the best one!
threshold_scan = np.linspace(0,1,11)
coefs = []
rmse = mean_squared_error(x_train, np.zeros(x_train.shape), squared=False)
x_train_added_noise = x_train + np.random.normal(0, rmse / 10.0, x_train.shape)

for i, threshold in enumerate(threshold_scan):
    opt = ps.STLSQ(threshold=threshold)
    model = ps.SINDy(feature_names=model_names, optimizer=opt)
    model.fit(x_train_added_noise, t=dt)
    coefs.append(model.coefficients())

plot_pareto(coefs, opt, model, threshold_scan, x_test, t_test)


In [ ]:
# Part 2a) differentiate (not with finite differences)

rmse = mean_squared_error(x_train, np.zeros(x_train.shape), squared=False)
x_train_added_noise = x_train + np.random.normal(0, rmse / 50.0, x_train.shape)

# with minor noise
plot_data_and_derivative(x_train_added_noise, dt, ps.SmoothedFiniteDifference()._differentiate)

In [ ]:
# Part 2b) add more data

n_trajectories = 40
x0s = (np.random.randn(n_trajectories, 3) -0.5) * 20
x_train_multi = []
for i in range(n_trajectories):
    x_train_temp = solve_ivp(lorenz, t_train_span, x0s[i]),
                            t_eval=t_train, **integrator_keyword).y.T
    rmse = mean_squared_error(x_train, np.zeros(x_train.shape), squared=False)
    x_train_multi.append(x_train + np.random.normal(0, rmse / 50.0, x_train.shape))

    model = ps.SINDy(feature_names=feature_names, optimizer=ps.STLSQ())
    model.fit(x_train_multi, t=dt, multiple_trajectories=True)
    model.print()


In [ ]:
# Part 2c) make different models for different subsamples


rmse = mean_squared_error(x_train, np.zeros(x_train.shape), squared=False)
x_train_added_noise = x_train + np.random.normal(0, rmse / 10.0, x_train.shape)

opt = ps.STLSQ(threshold=0.5)
model = ps.SINDy(feature_names=model_names, optimizer=opt)
model.fit(x_train_added_noise, t=dt)
model.print()

# ensambled many models
model.fit(x_train_added_noise, t=dt, ensemble=True)
mean_ensemble = np.mean(model.coef_list, axis=0)
std_ensemble = np.std(model.coef_list, axis=0)

model.fit(x_train_added_noise, t=dt, library_ensemble=True)
mean_library_ensemble = np.mean(model.coef_list, axis=0)
std_library_ensemble = np.std(model.coef_list, axis=0)

plot_ensemble_results(model, mean_ensemble, std_ensemble, mean_library_ensemble, std_library_ensemble)

In [ ]:
# Part 2d), e), f) - advanced (tutorial 5) - constrain model, trapping SINDy, weak formulation

In [ ]:
# Part 3 - PDEs

# data - to be defined
t = 1
x = 5
u = 1000000
u_dot = 23323

u_flattened = np.reshape(u, (len(x)*len(t), 1))
u_dot_flattened = np.reshape(u_dot, (len(x)*len(t), 1))

library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]

pde_lib = ps.PDELibrary(library_functions=library_functions, function_names=library_function_names, derivative_order=4, spatial_grid=x, is_uniform=True)
pde_opt = ps.STLSQ(threshold=30, normalize_columns=True)
model = py.SINDy(optimizer=pde_opt, feature_library=pde_lib, feature_names=['u'])
model.fit(u_flattened, x_dot=u_dot_flattened)
model.print()


$u_t = - u_{xxxx} - uu_x - u_{xx}$